In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm
tqdm.pandas()
from catboost import Pool, CatBoostClassifier

In [2]:
data = pd.read_csv('D:/备份/0123_busno数据.csv',low_memory=False)
data = data[data['busno'] != 999]
data['enstation_time'] = pd.to_datetime(data['enstation_time'])
data['day'] = data['enstation_time'].dt.day
data['hour'] = data['enstation_time'].dt.hour
data['minute'] = data['enstation_time'].dt.minute
# data['second'] = data['enstation_time'].dt.second
data = data[data['enstation_name'] != data['exstation_name']]
data.dropna(subset='card_type',inplace=True)
data['card_type'] = data['card_type'].astype('int')
data['busno'] = data['busno'].astype('int')
data = data[data['card_no'] != 999]
data['card_no'] = data['card_no'].astype('str')

In [3]:
cat_fea = ['card_no','line_name','card_type','busno']

In [4]:
data['exstation_district'] = data['exstation_street'].str[:3]
data['enstation_district'] = data['enstation_street'].str[:3]

In [5]:
data

,card_no,enstation_time,enstation_name,orientation,line_name,lon1,lat1,exstation_name,lon2,lat2,...,exstation_street,bus_lenth,station_num,card_type,busno,day,hour,minute,exstation_district,enstation_district
0,1000000000060373.0,2022-11-03 18:12:15,新鸿路,2.0,71,104.10720,30.661700,小龙桥路,104.110880,30.659771,...,成华区新鸿路街道,32,16.0,69,21042442,3,18,12,成华区,成华区
1,1000000000060776.0,2022-11-03 07:24:17,华阳客运站,1.0,571,104.06820,30.487530,龙泉汽车总站,104.256395,30.562378,...,龙泉驿区龙泉街道,36,0.0,69,21071207,3,7,24,龙泉驿,双流区
2,1000000000060995.0,2022-11-03 07:13:01,文家大道南,1.0,232,103.94210,30.693420,文家大道成温路口,103.940245,30.689269,...,青羊区文家街道,16,9.0,69,21025017,3,7,13,青羊区,青羊区
3,1000000000061081.0,2022-11-03 08:02:53,凯德风尚,1.0,309,103.95750,30.668860,光华大道一段东二站,103.952034,30.668457,...,青羊区文家街道,42,8.0,69,21143292,3,8,2,青羊区,青羊区
4,1000000000062150.0,2022-11-03 17:49:07,繁清家具厂,2.0,316,104.03617,30.796137,新泉路口,104.042276,30.715166,...,金牛区沙河源街道,33,21.0,69,21042374,3,17,49,金牛区,新都区
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9318126,9999703015237632.0,2022-11-15 08:58:41,中和二街,1.0,259,104.09340,30.539770,地铁天府三街站,104.069570,30.547350,...,武侯区桂溪街道,12,5.0,74,21039352,15,8,58,武侯区,双流区
9318129,3.104870390304682e+18,2022-11-15 07:41:04,电子科大清水河校区南校门,2.0,205,103.92440,30.743910,电子科大清水河校区西校门,103.926867,30.755832,...,郫都区合作街道,18,2.0,32,21028631,15,7,41,郫都区,郫都区
9318131,3.1051700701090115e+18,2022-11-15 08:26:48,致强环街,1.0,71,104.11590,30.700920,二仙桥西路,104.125499,30.681027,...,成华区桃蹊路街道,32,5.0,32,21042502,15,8,26,成华区,成华区
9318132,3.105170070114288e+18,2022-11-15 09:03:56,安靖快铁公交站,1.0,143,104.03520,30.736160,地铁金华寺东路站,104.124843,30.769408,...,新都区三河街道,29,13.0,32,21022085,15,9,3,新都区,郫都区


In [7]:
data = data[(data['exstation_district'] == '武侯区') | (data['exstation_district'] == '锦江区') |
     (data['exstation_district'] == '青羊区') | (data['exstation_district'] == '金牛区') |
     (data['exstation_district'] == '成华区')].reset_index(drop=True)
data = data[(data['enstation_district'] == '武侯区') | (data['enstation_district'] == '锦江区') |
     (data['enstation_district'] == '青羊区') | (data['enstation_district'] == '金牛区') |
     (data['enstation_district'] == '成华区')].reset_index(drop=True)

In [8]:
data

,card_no,enstation_time,enstation_name,orientation,line_name,lon1,lat1,exstation_name,lon2,lat2,...,exstation_street,bus_lenth,station_num,card_type,busno,day,hour,minute,exstation_district,enstation_district
0,1000000000060373.0,2022-11-03 18:12:15,新鸿路,2.0,71,104.1072,30.66170,小龙桥路,104.110880,30.659771,...,成华区新鸿路街道,32,16.0,69,21042442,3,18,12,成华区,成华区
1,1000000000060995.0,2022-11-03 07:13:01,文家大道南,1.0,232,103.9421,30.69342,文家大道成温路口,103.940245,30.689269,...,青羊区文家街道,16,9.0,69,21025017,3,7,13,青羊区,青羊区
2,1000000000061081.0,2022-11-03 08:02:53,凯德风尚,1.0,309,103.9575,30.66886,光华大道一段东二站,103.952034,30.668457,...,青羊区文家街道,42,8.0,69,21143292,3,8,2,青羊区,青羊区
3,1000000000062276.0,2022-11-03 17:43:10,西御街,1.0,13,104.0627,30.65633,清波社区,103.986536,30.681534,...,青羊区苏坡街道,26,24.0,69,21022425,3,17,43,青羊区,青羊区
4,1000000000063548.0,2022-11-03 07:34:37,清波社区,2.0,228,103.9853,30.68277,羊犀立交南,104.003737,30.698363,...,金牛区金泉街道,22,8.0,69,21022181,3,7,34,金牛区,青羊区
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6314851,9963941808023552.0,2022-11-15 08:40:43,新成彭路口,2.0,25,104.0473,30.72058,川陕路口,104.098562,30.707368,...,成华区双水碾街道,30,11.0,69,21042275,15,8,40,成华区,金牛区
6314852,9986588534548480.0,2022-11-15 12:42:37,上沙河铺路,2.0,47,104.1229,30.62358,地铁东大路站,104.111689,30.633912,...,锦江区双桂路街道,39,7.0,69,21012416,15,12,42,锦江区,锦江区
6314853,9994650285156352.0,2022-11-15 09:39:54,二环成仁公交站,2.0,120,104.0981,30.62448,地铁槐树店站,104.138256,30.648232,...,成华区万年场街道,29,14.0,69,21012078,15,9,39,成华区,锦江区
6314854,3.1051700701090115e+18,2022-11-15 08:26:48,致强环街,1.0,71,104.1159,30.70092,二仙桥西路,104.125499,30.681027,...,成华区桃蹊路街道,32,5.0,32,21042502,15,8,26,成华区,成华区


In [10]:
line_names = data['line_name'].unique()

In [12]:
for line_name in tqdm(line_names[90:]):
    try:
        data_tem = data[data['line_name'] == line_name].reset_index(drop=True)
        y_data = data_tem['exstation_name']
        x_data = data_tem.drop(['exstation_street','exstation_name','lon2','lat2','enstation_time',
                    'enstation_name','exstation_district','enstation_street','enstation_district'],axis=1)
        X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)
        train_dataset = Pool(data=X_train,label=y_train,cat_features=cat_fea)
        eval_dataset = Pool(data=X_test,label=y_test,cat_features=cat_fea)
        model = CatBoostClassifier(loss_function='MultiClass',task_type='GPU',iterations=5000,eval_metric='Accuracy',
                           early_stopping_rounds=50)
        model.fit(train_dataset,eval_set=eval_dataset,verbose=False)
        model.save_model('0401CatBoost模型/' + line_name + '路模型.json')

    except Exception as e:
        print(f'{line_name}路模型训练失败')
        print(e)

  3%|██                                                                             | 11/420 [03:50<1:38:10, 14.40s/it]

G503路模型训练失败
C:/Go_Agent/pipelines/BuildMaster/catboost.git/catboost/private/libs/algo/data.cpp:196: Dataset test #0 contains class label "\xE5\xA4\xA9\xE5\xBA\x9C\xE4\xB8\x89\xE8\xA1\x97\xE5\x90\x89\xE6\xB3\xB0\xE8\xB7\xAF\xE5\x8F\xA3" that is not present in the learn dataset


 52%|█████████████████████████████████████████▎                                      | 217/420 [35:38<17:43,  5.24s/it]

G88路模型训练失败
C:/Go_Agent/pipelines/BuildMaster/catboost.git/catboost/private/libs/algo/data.cpp:196: Dataset test #0 contains class label "\xE4\xBA\xA4\xE5\xA4\xA7\xE7\xAB\x8B\xE4\xBA\xA4\xE4\xB8\x9C" that is not present in the learn dataset


 55%|████████████████████████████████████████████                                    | 231/420 [36:46<13:28,  4.28s/it]

...

332A路模型训练失败
C:/Go_Agent/pipelines/BuildMaster/catboost.git/catboost/libs/data/quantization.cpp:2416: All features are either constant or ignored.


100%|███████████████████████████████████████████████████████████████████████████████▊| 419/420 [47:51<00:01,  1.04s/it]

D705路模型训练失败
With n_samples=1, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.


100%|████████████████████████████████████████████████████████████████████████████████| 420/420 [47:51<00:00,  6.84s/it]

G166路模型训练失败
With n_samples=1, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.


In [16]:
from sklearn.metrics import classification_report
import numpy as np
import os

In [17]:
file_names = os.listdir('0401CatBoost模型/')
line_names = [file_name.replace("路模型.json",'') for file_name in file_names]

In [18]:
out = pd.DataFrame()
accuracy = np.empty(0)
precision = np.empty(0)
recall = np.empty(0)
f1 = np.empty(0)
ln = np.empty(0)
for line_name in tqdm(line_names):
    df_tem_UTC = data[data['line_name'] == f'{line_name}'].reset_index(drop=True)
    x_data = df_tem_UTC.drop(['exstation_street','exstation_name','lon2','lat2','enstation_time',
                    'enstation_name','exstation_district','enstation_street','enstation_district'],axis=1)
    y_data = df_tem_UTC['exstation_name']
    model = CatBoostClassifier().load_model('C:/Users/HLA_3D_DEV/Documents/LJY0226/0401CatBoost模型/' + line_name + '路模型.json')
    X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)
    y_pre = model.predict(X_test)
    
    report_dict = classification_report(y_test,y_pre,output_dict=True)
    report_df = pd.DataFrame(report_dict).transpose()
    accuracy = np.append(accuracy,report_df.iloc[-3]['support'])
    precision = np.append(precision,report_df.iloc[-1]['precision'])
    recall = np.append(recall,report_df.iloc[-1]['recall'])
    f1 = np.append(f1,report_df.iloc[-1]['f1-score'])
    ln = np.append(ln,line_name)
out = pd.DataFrame({'line_name':ln,'accuracy':accuracy,'precision':precision,'reall':recall,'f1':f1})

  0%|                                                                                          | 0/492 [00:00<?, ?it/s]C:\Users\HLA_3D_DEV\miniconda3\envs\Catboost\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HLA_3D_DEV\miniconda3\envs\Catboost\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HLA_3D_DEV\miniconda3\envs\Catboost\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predic

  2%|█▋                                                                               | 10/492 [00:04<03:39,  2.20it/s]C:\Users\HLA_3D_DEV\miniconda3\envs\Catboost\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HLA_3D_DEV\miniconda3\envs\Catboost\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HLA_3D_DEV\miniconda3\envs\Catboost\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predic

  4%|██▉                                                                              | 18/492 [00:07<03:20,  2.37it/s]C:\Users\HLA_3D_DEV\miniconda3\envs\Catboost\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HLA_3D_DEV\miniconda3\envs\Catboost\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HLA_3D_DEV\miniconda3\envs\Catboost\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predic

  5%|████▍                                                                            | 27/492 [00:11<03:14,  2.39it/s]C:\Users\HLA_3D_DEV\miniconda3\envs\Catboost\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HLA_3D_DEV\miniconda3\envs\Catboost\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HLA_3D_DEV\miniconda3\envs\Catboost\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predic

  7%|█████▊                                                                           | 35/492 [00:14<03:08,  2.43it/s]C:\Users\HLA_3D_DEV\miniconda3\envs\Catboost\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HLA_3D_DEV\miniconda3\envs\Catboost\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HLA_3D_DEV\miniconda3\envs\Catboost\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predic

...

 94%|███████████████████████████████████████████████████████████████████████████     | 462/492 [04:28<00:14,  2.10it/s]C:\Users\HLA_3D_DEV\miniconda3\envs\Catboost\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HLA_3D_DEV\miniconda3\envs\Catboost\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HLA_3D_DEV\miniconda3\envs\Catboost\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predic

 95%|████████████████████████████████████████████████████████████████████████████▎   | 469/492 [04:31<00:10,  2.24it/s]C:\Users\HLA_3D_DEV\miniconda3\envs\Catboost\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HLA_3D_DEV\miniconda3\envs\Catboost\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HLA_3D_DEV\miniconda3\envs\Catboost\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predic

 97%|█████████████████████████████████████████████████████████████████████████████▌  | 477/492 [04:35<00:08,  1.87it/s]C:\Users\HLA_3D_DEV\miniconda3\envs\Catboost\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HLA_3D_DEV\miniconda3\envs\Catboost\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HLA_3D_DEV\miniconda3\envs\Catboost\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samp

 98%|██████████████████████████████████████████████████████████████████████████████▋ | 484/492 [04:39<00:05,  1.44it/s]C:\Users\HLA_3D_DEV\miniconda3\envs\Catboost\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HLA_3D_DEV\miniconda3\envs\Catboost\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HLA_3D_DEV\miniconda3\envs\Catboost\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predic

100%|███████████████████████████████████████████████████████████████████████████████▋| 490/492 [04:42<00:00,  2.08it/s]C:\Users\HLA_3D_DEV\miniconda3\envs\Catboost\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HLA_3D_DEV\miniconda3\envs\Catboost\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HLA_3D_DEV\miniconda3\envs\Catboost\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predic

In [19]:
out

,line_name,accuracy,precision,reall,f1
0,1001,0.319734,0.321868,0.319734,0.288539
1,1002,0.619247,0.637328,0.619247,0.531386
2,1003,0.417925,0.441412,0.417925,0.373072
3,1004,0.475458,0.423133,0.475458,0.408939
4,1005,0.346760,0.309140,0.346760,0.302852
...,...,...,...,...,...
487,K7,0.726708,0.689811,0.726708,0.704852
488,P45,0.000000,0.000000,0.000000,0.000000
489,P53,0.333333,0.333333,0.333333,0.333333
490,T203,0.559322,0.479177,0.559322,0.495369


In [20]:
for keys in out.columns[1:]:
    print(f'{keys}: {np.average(out[keys])}')

accuracy: 0.4613763201194644
precision: 0.454036319453914
reall: 0.4613763201194644
f1: 0.4238549314204468


# 预测

In [21]:
df = pd.read_csv('D:/备份/0226UTC.csv',low_memory=False)

In [22]:
data = df.copy()
data['enstation_time'] = pd.to_datetime(data['enstation_time'])
data = data[data['busno'] != 999]
data['enstation_time'] = pd.to_datetime(data['enstation_time'])
data['day'] = data['enstation_time'].dt.day
data['hour'] = data['enstation_time'].dt.hour
data['minute'] = data['enstation_time'].dt.minute
data.dropna(subset='card_type',inplace=True)
data['card_type'] = data['card_type'].astype('int')
data['busno'] = data['busno'].astype('int')
data = data[data['card_no'] != 999]
data['card_no'] = data['card_no'].astype('str')

In [23]:
file_names = os.listdir('0401CatBoost模型/')
line_names = [file_name.replace("路模型.json",'') for file_name in file_names]

In [24]:
CatBoostClassifier().load_model('0401CatBoost模型/1001路模型.json').feature_names_

['card_no',
 'orientation',
 'line_name',
 'lon1',
 'lat1',
 'bus_lenth',
 'station_num',
 'card_type',
 'busno',
 'day',
 'hour',
 'minute']

In [30]:
data['station_num'] = data['station_num'].astype('int')

In [34]:
x_data

,card_no,orientation,line_name,lon1,lat1,bus_lenth,station_num,card_type,busno,day,hour,minute
0,5087650737949696.0,1,1001,104.1160,30.65345,14.0,3,74,21012190,3,16,56
1,5269805000000000.0,1,1001,104.1142,30.65731,14.0,4,74,21012005,3,19,17
2,6100000100002264.0,1,1001,104.1209,30.66063,14.0,0,15,21015016,3,16,10
3,6100000100066949.0,1,1001,104.1155,30.65734,14.0,10,15,21012190,3,11,29
4,6100000100150293.0,1,1001,104.1070,30.65744,14.0,6,15,21012005,3,12,38
...,...,...,...,...,...,...,...,...,...,...,...,...
11251,6100000161069008.0,1,1001,104.1074,30.66169,14.0,8,15,21012005,16,18,52
11252,6100000114853908.0,1,1001,104.1074,30.66169,14.0,8,15,21012005,16,18,52
11253,6100100080057620.0,1,1001,104.1192,30.65805,14.0,1,15,21012192,16,18,53
11254,6100000112661203.0,1,1001,104.1137,30.66177,14.0,9,15,21012005,16,18,54


In [36]:
out = []
for line_name in tqdm(line_names):
    data_tem_UTC = data[data['line_name'] == f'{line_name}'].reset_index(drop=True)
    df_tem_UTC = df[df['line_name'] == f'{line_name}'].reset_index(drop=True)
    x_data = data_tem_UTC[['card_no', 'orientation','line_name',
               'lon1', 'lat1','bus_lenth', 'station_num', 'card_type','busno' ,'day', 'hour', 'minute']]
    model = CatBoostClassifier().load_model('C:/Users/HLA_3D_DEV/Documents/LJY0226/0401CatBoost模型/' + line_name + '路模型.json')
    y_data = model.predict(x_data)
    df_tem_UTC['exstation_name'] = y_data.reshape(-1)
    out.append(df_tem_UTC)

100%|████████████████████████████████████████████████████████████████████████████████| 492/492 [11:16<00:00,  1.37s/it]


In [37]:
out = pd.concat(out,ignore_index=True)

In [38]:
out.to_csv('0401CatBoost_5.csv',index=False)